# 🗄️ Module 1: Core Python & Data - Week 3 Lecture 15
**Date:** 05/09/2025  
**Ducomented by:** Muhammad Soban Shaukat

## 🔗 Beyond Basics: SQL Joins and Aggregation

Welcome to Lecture 15! In our previous session, we learned the fundamentals of SQL and relational databases. Today, we'll dive deeper into one of the most powerful features of SQL: **JOIN operations** and **data aggregation**. These concepts allow us to combine data from multiple tables and generate meaningful insights from our data.

```python
# Setup: Import necessary libraries and create a sample database
import sqlite3
import pandas as pd

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create sample tables
cursor.executescript('''
CREATE TABLE Suppliers (
    SupplierID INTEGER PRIMARY KEY,
    SupplierName TEXT,
    Country TEXT
);

CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Price REAL,
    SupplierID INTEGER,
    FOREIGN KEY (SupplierID) REFERENCES Suppliers(SupplierID)
);

CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT,
    City TEXT,
    Country TEXT
);

CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    OrderDate TEXT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

# Insert sample data
cursor.executescript('''
INSERT INTO Suppliers VALUES
(1, 'Exotic Liquids', 'UK'),
(2, 'New Orleans Cajun Delights', 'USA'),
(3, 'Grandma Kelly''s Homestead', 'USA'),
(4, 'Tokyo Traders', 'Japan'),
(5, 'Cooperativa de Quesos ''Las Cabras''', 'Spain'),
(6, 'Inactive Supplier', 'France');  -- Supplier with no products

INSERT INTO Products VALUES
(1, 'Chai', 18.00, 1),
(2, 'Chang', 19.00, 1),
(3, 'Aniseed Syrup', 10.00, 1),
(4, 'Chef Anton''s Cajun Seasoning', 22.00, 2),
(5, 'Chef Anton''s Gumbo Mix', 21.35, 2),
(6, 'Grandma''s Boysenberry Spread', 25.00, 3),
(7, 'Uncle Bob''s Organic Dried Pears', 30.00, NULL),  -- Product with no supplier
(8, 'Northwoods Cranberry Sauce', 40.00, 3);

INSERT INTO Customers VALUES
(1, 'Alfreds Futterkiste', 'Berlin', 'Germany'),
(2, 'Ana Trujillo', 'México D.F.', 'Mexico'),
(3, 'Antonio Moreno', 'México D.F.', 'Mexico'),
(4, 'Around the Horn', 'London', 'UK'),
(5, 'Berglunds snabbköp', 'Luleå', 'Sweden');

INSERT INTO Orders VALUES
(101, 1, '2025-09-01'),
(102, 2, '2025-09-02'),
(103, 1, '2025-09-03'),
(104, 3, '2025-09-04'),
(105, 1, '2025-09-05');
''')

conn.commit()

print("✅ Sample database created with:")
print("   - Suppliers table (6 records)")
print("   - Products table (8 records)")
print("   - Customers table (5 records)")
print("   - Orders table (5 records)")
```

## 📋 Today's Agenda

1.  **🤔 The "Why" - The Problem with Separate Data**
    *   The Warehouse Manager's Dilemma

2.  **🔗 Core Concepts of Joining Tables**
    *   Understanding Table Aliases
    *   INNER JOIN: Finding Perfect Matches
    *   LEFT JOIN: Finding What's Missing
    *   RIGHT JOIN: The Opposite View
    *   FULL JOIN: The Complete Audit

3.  **📊 Exploring Aggregation**
    *   GROUP BY & HAVING clauses

4.  **🧪 Putting It All Together & Hands-On Lab**

---

## 1. 🤔 The "Why" - The Problem with Separate Data

### 📊 The Warehouse Manager's Dilemma

Imagine you're managing a warehouse with two separate record systems:
- A clipboard with product names and supplier IDs
- A massive binder with supplier details

To find who supplies "Chai Tea," you'd need to:
1. Find the product on your clipboard and note its SupplierID
2. Flip through the entire binder to find the supplier with that ID

This process is slow, inefficient, and prone to error!

```python
# Demonstrate the inefficient approach
print("📋 PRODUCTS CLIPBOARD:")
products_clipboard = [
    {"ProductName": "Chai", "SupplierID": 1},
    {"ProductName": "Chang", "SupplierID": 1},
    {"ProductName": "Aniseed Syrup", "SupplierID": 1}
]

print(f"{'Product':<20} {'SupplierID':<12}")
print("-" * 32)
for product in products_clipboard:
    print(f"{product['ProductName']:<20} {product['SupplierID']:<12}")

print("\n📚 SUPPLIERS BINDER:")
suppliers_binder = [
    {"SupplierID": 1, "SupplierName": "Exotic Liquids", "Phone": "123-456-7890"},
    {"SupplierID": 2, "SupplierName": "New Orleans Cajun Delights", "Phone": "234-567-8901"},
    {"SupplierID": 3, "SupplierName": "Grandma Kelly's Homestead", "Phone": "345-678-9012"}
]

print(f"{'SupplierID':<12} {'SupplierName':<30} {'Phone':<15}")
print("-" * 57)
for supplier in suppliers_binder:
    print(f"{supplier['SupplierID']:<12} {supplier['SupplierName']:<30} {supplier['Phone']:<15}")

print("\n🔍 MANUAL LOOKUP PROCESS:")
print("1. Find 'Chai' on clipboard → SupplierID: 1")
print("2. Search binder for SupplierID 1 → Exotic Liquids")
print("3. Contact: 123-456-7890")
print("\n❌ This process is SLOW and ERROR-PRONE!")
```

---

## 2. 🔗 Core Concepts of Joining Tables

### 🏗️ The Relational Solution: SQL JOINS

SQL JOINs are the efficient solution to the warehouse manager's dilemma. They allow us to combine data from multiple tables in a single query.

```python
# Show the power of JOINs
print("💪 THE POWER OF SQL JOINS")
print("=" * 25)

# INNER JOIN example
query = '''
SELECT P.ProductName, S.SupplierName, S.Country
FROM Products P
INNER JOIN Suppliers S ON P.SupplierID = S.SupplierID
WHERE P.ProductName = 'Chai'
'''

result = pd.read_sql_query(query, conn)
print("🔍 FINDING SUPPLIER FOR 'CHAI':")
print(result.to_string(index=False))
print("\n✅ Single query gives us all the information we need!")
```

### 📛 Understanding Table Aliases

Table aliases (like `P` for Products, `S` for Suppliers) make queries shorter and more readable. They're essential when joining tables with columns of the same name.

```python
# Demonstrate table aliases
print("📛 TABLE ALIASES EXPLAINED")
print("=" * 30)

print("WITHOUT ALIASES (verbose):")
print("SELECT Products.ProductName, Suppliers.SupplierName")
print("FROM Products")
print("INNER JOIN Suppliers ON Products.SupplierID = Suppliers.SupplierID")
print()
print("WITH ALIASES (concise):")
print("SELECT P.ProductName, S.SupplierName")
print("FROM Products P")
print("INNER JOIN Suppliers S ON P.SupplierID = S.SupplierID")
print()
print("✅ Aliases make queries shorter and easier to read!")
```

---

## 3. 🔗 INNER JOIN: Finding Perfect Matches

### 📊 What is an INNER JOIN?

The INNER JOIN returns only the rows where there's a match in both tables. It's like finding the intersection of two datasets.

```python
# INNER JOIN example
print("🔗 INNER JOIN: PERFECT MATCHES")
print("=" * 35)

inner_join_query = '''
SELECT P.ProductName, S.SupplierName, S.Country
FROM Products P
INNER JOIN Suppliers S ON P.SupplierID = S.SupplierID
'''

inner_join_result = pd.read_sql_query(inner_join_query, conn)
print(inner_join_result.to_string(index=False))
print(f"\n📊 Total matches found: {len(inner_join_result)}")
print("✅ Only shows products that have valid suppliers!")
```

---

## 4. 🔗 LEFT JOIN: Finding What's Missing

### 📊 What is a LEFT JOIN?

A LEFT JOIN returns all rows from the left table (first table), and only the matched rows from the right table. If there's no match, the result is NULL on the right side.

```python
# LEFT JOIN example
print("🔗 LEFT JOIN: FINDING WHAT'S MISSING")
print("=" * 40)

left_join_query = '''
SELECT S.SupplierName, P.ProductName
FROM Suppliers S
LEFT JOIN Products P ON S.SupplierID = P.SupplierID
'''

left_join_result = pd.read_sql_query(left_join_query, conn)
print(left_join_result.to_string(index=False))
print(f"\n📊 Total suppliers: {len(left_join_result)}")

# Find inactive suppliers (those with no products)
inactive_suppliers = left_join_result[left_join_result['ProductName'].isnull()]
print(f"🚫 Inactive suppliers: {len(inactive_suppliers)}")
print(inactive_suppliers[['SupplierName']].to_string(index=False))
```

---

## 5. 🔗 RIGHT JOIN: The Opposite View

### 📊 What is a RIGHT JOIN?

A RIGHT JOIN returns all rows from the right table (second table), and only the matched rows from the left table. It's the mirror opposite of a LEFT JOIN.

**Note:** SQLite doesn't support RIGHT JOIN natively, but we can simulate it.

```python
# Simulating RIGHT JOIN in SQLite
print("🔗 RIGHT JOIN: THE OPPOSITE VIEW (Simulated)")
print("=" * 50)

right_join_query = '''
SELECT P.ProductName, S.SupplierName
FROM Products P
LEFT JOIN Suppliers S ON P.SupplierID = S.SupplierID
'''

right_join_result = pd.read_sql_query(right_join_query, conn)
print(right_join_result.to_string(index=False))
print(f"\n📊 Total products: {len(right_join_result)}")

# Find products with no supplier
orphan_products = right_join_result[right_join_result['SupplierName'].isnull()]
print(f"❌ Products with no supplier: {len(orphan_products)}")
print(orphan_products[['ProductName']].to_string(index=False))
```

---

## 6. 🔗 FULL JOIN: The Complete Audit

### 📊 What is a FULL JOIN?

A FULL JOIN (or FULL OUTER JOIN) combines LEFT and RIGHT JOINs. It returns all rows from both tables, matching them where possible. If there's no match, the columns from the other table will be NULL.

**Note:** SQLite doesn't support FULL JOIN natively, but we can simulate it.

```python
# Simulating FULL JOIN in SQLite
print("🔗 FULL JOIN: COMPLETE DATA AUDIT (Simulated)")
print("=" * 55)

full_join_query = '''
-- Part 1: Suppliers and their products (LEFT JOIN)
SELECT S.SupplierName, P.ProductName
FROM Suppliers S
LEFT JOIN Products P ON S.SupplierID = P.SupplierID

UNION

-- Part 2: Products with no supplier (RIGHT JOIN part)
SELECT S.SupplierName, P.ProductName
FROM Products P
LEFT JOIN Suppliers S ON P.SupplierID = S.SupplierID
WHERE S.SupplierID IS NULL
'''

full_join_result = pd.read_sql_query(full_join_query, conn)
print(full_join_result.to_string(index=False))
print(f"\n📊 Total records: {len(full_join_result)}")

# Analyze the results
suppliers_with_products = full_join_result[full_join_result['ProductName'].notnull() & full_join_result['SupplierName'].notnull()]
inactive_suppliers = full_join_result[full_join_result['ProductName'].isnull()]
orphan_products = full_join_result[full_join_result['SupplierName'].isnull()]

print(f"✅ Suppliers with products: {len(suppliers_with_products)}")
print(f"🚫 Inactive suppliers: {len(inactive_suppliers)}")
print(f"❌ Orphan products: {len(orphan_products)}")
```

---

## 7. 📊 Exploring Aggregation

### 🎯 The Need for Aggregation

While JOINs help us combine detailed data, aggregation helps us see the big picture by summarizing data.

```python
# Demonstrate the need for aggregation
print("📊 THE NEED FOR AGGREGATION")
print("=" * 30)

# Show raw orders data
orders_query = "SELECT * FROM Orders"
orders_result = pd.read_sql_query(orders_query, conn)
print("📋 RAW ORDERS DATA:")
print(orders_result.to_string(index=False))

print("\n🤔 BUSINESS QUESTIONS:")
print("   - How many orders per customer?")
print("   - Which customers order the most?")
print("   - Who are our most valuable customers?")
```

### 📊 GROUP BY Clause

The GROUP BY clause groups rows that have the same values into summary rows. It's used with aggregate functions like COUNT(), SUM(), AVG(), MAX(), MIN().

```python
# GROUP BY example
print("📊 GROUP BY: SUMMARIZING DATA")
print("=" * 35)

group_by_query = '''
SELECT C.CustomerName, COUNT(O.OrderID) AS NumberOfOrders
FROM Customers C
LEFT JOIN Orders O ON C.CustomerID = O.CustomerID
GROUP BY C.CustomerName
'''

group_by_result = pd.read_sql_query(group_by_query, conn)
print(group_by_result.to_string(index=False))
print("\n✅ Shows order count for each customer!")
```

### 📊 HAVING Clause

The HAVING clause filters the results of a GROUP BY based on the aggregate function's result. WHERE filters rows before grouping; HAVING filters groups after grouping.

```python
# HAVING example
print("📊 HAVING: FILTERING GROUPED RESULTS")
print("=" * 45)

having_query = '''
SELECT C.CustomerName, COUNT(O.OrderID) AS NumberOfOrders
FROM Customers C
LEFT JOIN Orders O ON C.CustomerID = O.CustomerID
GROUP BY C.CustomerName
HAVING COUNT(O.OrderID) > 1
'''

having_result = pd.read_sql_query(having_query, conn)
print(having_result.to_string(index=False))
print("\n✅ Shows only customers with more than 1 order!")
```

---

## 8. 🧪 Hands-On Lab: Advanced Queries

### 🎯 Practice Exercises

Let's apply what we've learned with some practical exercises.

```python
print("🧪 HANDS-ON LAB: ADVANCED QUERIES")
print("=" * 40)
```

**Exercise 1: Find products from a specific supplier**

```python
# Exercise 1: Products from 'Exotic Liquids'
print("1. 🔍 Find all products supplied by 'Exotic Liquids'")
print("=" * 55)

exotic_query = '''
SELECT P.ProductName, P.Price
FROM Products P
INNER JOIN Suppliers S ON P.SupplierID = S.SupplierID
WHERE S.SupplierName = 'Exotic Liquids'
'''

exotic_result = pd.read_sql_query(exotic_query, conn)
print(exotic_result.to_string(index=False))
print(f"\n✅ Found {len(exotic_result)} products from Exotic Liquids")
```

**Exercise 2: Count products per supplier**

```python
# Exercise 2: Products count per supplier
print("\n2. 📊 Count how many products each supplier provides")
print("=" * 60)

count_query = '''
SELECT S.SupplierName, COUNT(P.ProductID) AS NumberOfProducts
FROM Suppliers S
LEFT JOIN Products P ON S.SupplierID = P.SupplierID
GROUP BY S.SupplierName
'''

count_result = pd.read_sql_query(count_query, conn)
print(count_result.to_string(index=False))
```

**Exercise 3: Suppliers with more than 2 products**

```python
# Exercise 3: Suppliers with more than 2 products
print("\n3. 🏆 Find suppliers who provide more than 2 products")
print("=" * 65)

having_query = '''
SELECT S.SupplierName, COUNT(P.ProductID) AS NumberOfProducts
FROM Suppliers S
INNER JOIN Products P ON S.SupplierID = P.SupplierID
GROUP BY S.SupplierName
HAVING COUNT(P.ProductID) > 2
ORDER BY NumberOfProducts DESC
'''

having_result = pd.read_sql_query(having_query, conn)
print(having_result.to_string(index=False))
```

---

## 9. 🚀 Bonus Challenges

### 🎯 Advanced Practice Exercises

Test your skills with these bonus challenges!

```python
print("🚀 BONUS CHALLENGES")
print("=" * 20)
```

**Challenge 1: Finding Inactive Suppliers**

```python
# Challenge 1: Inactive suppliers
print("1. 🚫 Find suppliers who don't supply any products")
print("=" * 55)

inactive_query = '''
SELECT S.SupplierName
FROM Suppliers S
LEFT JOIN Products P ON S.SupplierID = P.SupplierID
WHERE P.ProductID IS NULL
'''

inactive_result = pd.read_sql_query(inactive_query, conn)
print(inactive_result.to_string(index=False))
print(f"\n✅ Found {len(inactive_result)} inactive suppliers")
```

**Challenge 2: Multi-Join and Group**

```python
# Challenge 2: Customers and their order counts
print("\n2. 👥 List all customers and their total orders")
print("=" * 50)

multi_join_query = '''
SELECT C.CustomerName, COUNT(O.OrderID) AS TotalOrders
FROM Customers C
LEFT JOIN Orders O ON C.CustomerID = O.CustomerID
GROUP BY C.CustomerName
ORDER BY TotalOrders DESC
'''

multi_join_result = pd.read_sql_query(multi_join_query, conn)
print(multi_join_result.to_string(index=False))
```

---

## 10. 📚 Comprehensive Summary

### 🎯 What We Learned Today:

1.  **SQL JOIN Types:**
    - INNER JOIN: Finds matches between tables
    - LEFT JOIN: Shows all records from left table + matches from right
    - RIGHT JOIN: Shows all records from right table + matches from left  
    - FULL JOIN: Shows all records from both tables

2.  **Aggregation:**
    - GROUP BY: Groups rows for summary calculations
    - HAVING: Filters grouped results
    - Aggregate functions: COUNT(), SUM(), AVG(), etc.

3.  **Practical Applications:**
    - Combining data from multiple tables
    - Finding missing data (inactive suppliers, orphan products)
    - Generating summary reports
    - Data quality auditing

### 🔑 Key SQL Syntax:

| Join Type | SQL Syntax | Purpose |
|:----------|:-----------|:--------|
| INNER JOIN | `SELECT ... FROM A INNER JOIN B ON A.id = B.id` | Find matching records |
| LEFT JOIN | `SELECT ... FROM A LEFT JOIN B ON A.id = B.id` | All from A + matches from B |
| RIGHT JOIN | `SELECT ... FROM A RIGHT JOIN B ON A.id = B.id` | All from B + matches from A |
| FULL JOIN | `SELECT ... FROM A FULL JOIN B ON A.id = B.id` | All records from both tables |

### 💡 Pro Tips:

1.  **Use aliases** for better readability
2.  **Test joins** with small datasets first
3.  **Be careful with NULL values** in outer joins
4.  **Use WHERE for row filtering**, HAVING for group filtering
5.  **Always specify join conditions** to avoid Cartesian products

### ✅ Completion Checklist:

- [x] Understand different types of SQL JOINs
- [x] Practice INNER, LEFT, RIGHT, and FULL JOINs
- [x] Learn to simulate RIGHT and FULL JOINs in SQLite
- [x] Master GROUP BY and HAVING clauses
- [x] Complete hands-on lab exercises
- [x] Solve bonus challenges

### 🔮 Next Steps:

1.  **Practice with real databases**: MySQL, PostgreSQL
2.  **Learn advanced SQL**: Subqueries, CTEs, window functions
3.  **Explore database optimization**: Indexes, query planning
4.  **Study database design**: Normalization, relationships
5.  **Learn about ORMs**: SQLAlchemy, Django ORM

```python
# Clean up
conn.close()
print("\n✅ Database connection closed")
print("🎉 Congratulations! You've mastered SQL JOINs and Aggregation!")
```
